In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
from contextlib import contextmanager


@contextmanager
def no_ssl_verify():
    import ssl
    from urllib import request

    try:
        request.urlopen.__kwdefaults__.update({'context': ssl.SSLContext()})
        yield
    finally:
        request.urlopen.__kwdefaults__.update({'context': None})

In [3]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michelecafagna26/gpt2-medium-finetuned-sst2-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("michelecafagna26/gpt2-medium-finetuned-sst2-sentiment")

In [15]:
import textattack
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [16]:
%cd /content/gdrive/MyDrive/Colab Notebooks/TextAttack/saved_results

/content/gdrive/MyDrive/Colab Notebooks/TextAttack/saved_results


In [10]:
# gpt2 mid sst2
import pickle
import pandas as pd
import datasets
import torch
import tqdm
import os

path = "/content/gdrive/MyDrive/Colab Notebooks/TextAttack/saved_results/pkls/"
files= os.listdir(path)
s = []

for file in files:
    with open('pkls/' + file, 'rb') as f:
        data = pickle.load(f)

    #print(data.keys())
    #print(data['train_labels'])
    print("####################"+file+"####################")

    ts = [i for i in data['train_sentences']]
    tl = [i for i in data['train_labels']]
    df = pd.DataFrame({'sentence':ts, 'label':tl})

    dataset_hf = datasets.Dataset.from_pandas(df)

    dataset = textattack.datasets.HuggingFaceDataset(dataset_hf)
    print(dataset_hf, dataset)
    attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=-1, checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=False)
    attacker = textattack.Attacker(attack, dataset, attack_args)
    attacker.attack_dataset()

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.


####################sst2_gpt2-xl_4shot_1000_subsample_seed0.pkl####################
Dataset({
    features: ['sentence', 'label'],
    num_rows: 4
}) <textattack.datasets.huggingface_dataset.HuggingFaceDataset object at 0x7f543f468e20>


100%|██████████| 481M/481M [00:38<00:00, 12.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp2mu1ro7_.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.gpt2.modeling_gpt2.GPT2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

 25%|██▌       | 1/4 [00:54<02:43, 54.59s/it]

...

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[0 (69%)]]

it offers a [[glimpse]] of the solomonic [[decision]] [[facing]] jewish parents in those [[turbulent]] [[times]] : to [[save]] their children and yet to lose them .

it offers a [[chronically]] of the solomonic [[stoppage]] [[braved]] jewish parents in those [[unsteady]] [[timetables]] : to [[bailout]] their children and yet to lose them .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 25.0%  |
| Average num. words per input: | 24.0   |
| Avg num queries:              | 192.0  |
+-------

textattack: Unknown if model of class <class 'transformers.models.gpt2.modeling_gpt2.GPT2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


####################sst2_gpt2-xl_1shot_1000_subsample_seed4.pkl####################
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1
}) <textattack.datasets.huggingface_dataset.HuggingFaceDataset object at 0x7f5410729fa0>
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_sho

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1: 100%|██████████| 1/1 [00:08<00:00,  8.37s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (100%)]] --> [[1 (100%)]]

it feels like very light errol morris , focusing on eccentricity but [[failing]] , ultimately , to make something bigger out of its scrapbook of oddballs .

it feels like very light errol morris , focusing on eccentricity but [[overlooking]] , ultimately , to make something bigger out of its scrapbook of oddballs .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 4.35%  |
| Average num. words per input: | 23.0   |
| Avg num queries:              | 63.0   |
+-------------------------------

textattack: Unknown if model of class <class 'transformers.models.gpt2.modeling_gpt2.GPT2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Dataset({
    features: ['sentence', 'label'],
    num_rows: 8
}) <textattack.datasets.huggingface_dataset.HuggingFaceDataset object at 0x7f7a36ba8d60>
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3):

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  12%|█▎        | 1/8 [00:16<01:54, 16.31s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[0 (69%)]]

it offers a [[glimpse]] of the solomonic [[decision]] [[facing]] jewish parents in those [[turbulent]] [[times]] : to [[save]] their children and yet to lose them .

it offers a [[chronically]] of the solomonic [[stoppage]] [[braved]] jewish parents in those [[unsteady]] [[timetables]] : to [[bailout]] their children and yet to lose them .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  25%|██▌       | 2/8 [00:23<01:09, 11.59s/it]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[1 (100%)]] --> [[0 (100%)]]

any movie that makes hard work seem heroic [[deserves]] a [[look]] .

any movie that makes hard work seem heroic [[justifies]] a [[blankly]] .




[Succeeded / Failed / Skipped / Total] 7 / 0 / 0 / 7:  88%|████████▊ | 7/8 [01:05<00:09,  9.38s/it]

--------------------------------------------- Result 7 ---------------------------------------------
[[1 (100%)]] --> [[0 (100%)]]

rifkin 's references are ... [[impeccable]] throughout .

rifkin 's references are ... [[unsullied]] throughout .




[Succeeded / Failed / Skipped / Total] 8 / 0 / 0 / 8: 100%|██████████| 8/8 [01:07<00:00,  8.39s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (100%)]] --> [[0 (80%)]]

world traveler might not go anywhere new , or arrive anyplace special , but it 's certainly an [[honest]] attempt to get at something .

world traveler might not go anywhere new , or arrive anyplace special , but it 's certainly an [[franc]] attempt to get at something .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 20.12% |
| Average num. words per input: | 20.38  |
| Avg num queries:              | 130.38 |
+-------------------------------+--------+
